<a href="https://colab.research.google.com/github/jesuegraciliano/prompts/blob/main/sepei_oficina.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29

INFO: pip is looking at multiple versions of embedchain to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of embedchain to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of embedchain[github,youtube] to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of embedchain[github,youtube] to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 160.7/160.7 kB 4.1 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of langchain to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of langchain-cohere to determine which version is compatible with other requirements. This could t

In [1]:
import os
from crewai_tools import tool, SerperDevTool, ScrapeWebsiteTool, WebsiteSearchTool
from crewai_tools import tool
from crewai import Agent, Task, Crew, Process
from langchain_openai import ChatOpenAI

In [2]:
# Função para ler a chave de API do arquivo
def get_openai_api_key():
    with open('API_OpenAI_SEPEI.txt', 'r') as file:
        return file.read().strip()

In [3]:
# Função para ler a chave da API Serper
def get_serper_api_key():
    with open('API_Serper.txt', 'r') as file:
        return file.read().strip()

In [4]:
# Configurar a chave de API como variável de ambiente
openai_api_key = get_openai_api_key()
serper_api_key = get_serper_api_key()
os.environ["OPENAI_API_KEY"] = openai_api_key
os.environ["SERPER_API_KEY"] = serper_api_key


In [5]:
gpt4o_mini_llm = ChatOpenAI(model="gpt-4o-mini", api_key=openai_api_key)

In [6]:
# Ferramenta para busca no google
ferramenta_de_busca = SerperDevTool()

# Ferramenta para raspagem de sites
ferramenta_de_raspagem = ScrapeWebsiteTool()

In [7]:
planejador = Agent(
    role="Planejador de Conteúdo",
    goal="Planejar conteúdo envolvente e factualmente preciso sobre {tópico}",
    backstory="Você está trabalhando no planejamento de um artigo de blog "
              "sobre o tópico: {tópico}."
              "Você coleta informações que ajudam o "
              "público a aprender algo "
              "e tomar decisões informadas. "
              "Seu trabalho é a base para que o "
              "Redator de Conteúdo escreva um artigo sobre este tópico.",
    verbose=True,
    tools=[ferramenta_de_busca, ferramenta_de_raspagem],
    allow_delegation=False,
    llm=gpt4o_mini_llm
)

In [8]:
escritor = Agent(
    role="Redator de Conteúdo",
    goal="Escrever um artigo, em português do Brasil, de opinião perspicaz e factualmente preciso "
         "sobre o tópico: {tópico}",
    backstory="Você está trabalhando na escrita de um "
              "novo artigo de opinião sobre o tópico: {tópico}. "
              "Você baseia sua escrita no trabalho do "
              "Planejador de Conteúdo, que fornece um esboço "
              "e contexto relevante sobre o tópico. "
              "Você segue os principais objetivos e "
              "a direção do esboço, "
              "conforme fornecido pelo Planejador de Conteúdo. "
              "Você também fornece insights objetivos e imparciais "
              "e os apoia com as informações "
              "fornecidas pelo Planejador de Conteúdo. "
              "Você reconhece em seu artigo de opinião "
              "quando suas declarações são opiniões "
              "em vez de afirmações objetivas.",
    verbose=True,
    tools=[ferramenta_de_busca, ferramenta_de_raspagem],
    allow_delegation=False,
    llm=gpt4o_mini_llm
)

In [9]:
editor = Agent(
    role="Editor",
    goal="Editar um post para um blog alinhado com "
         "o estilo de escrita da organização.",
    backstory="Você é um editor que recebe um post de blog "
              "do Redator de Conteúdo. "
              "Seu objetivo é revisar o post de blog "
              "para garantir que ele siga as melhores práticas jornalísticas, "
              "forneça pontos de vista equilibrados "
              "ao apresentar opiniões ou afirmações, "
              "e também evite tópicos ou opiniões "
              "altamente controversos sempre que possível.",
    verbose=True,
    tools=[ferramenta_de_busca, ferramenta_de_raspagem],
    allow_delegation=False,
    llm=gpt4o_mini_llm
)


In [10]:
planejamento = Task(
    description=(
        "1. Priorizar as últimas tendências, principais players "
            "e notícias relevantes sobre {tópico}.\n"
        "2. Identificar o público-alvo, considerando "
            "seus interesses e pontos de dor.\n"
        "3. Desenvolver um esboço de conteúdo detalhado, incluindo "
            "uma introdução, pontos principais e um chamado para ação.\n"
        "4. Incluir palavras-chave de SEO e dados ou fontes relevantes."
    ),
    expected_output="Um documento abrangente de plano de conteúdo "
        "com um esboço, análise de público-alvo, "
        "palavras-chave de SEO e demais recursos.",
    agent=planejador,
)


In [11]:
escrita = Task(
    description=(
        "1. Use o plano de conteúdo para criar um "
            "post de blog atraente sobre {tópico}.\n"
        "2. Incorpore palavras-chave de SEO de forma natural.\n"
		    "3. As seções/subtítulos devem ser nomeados "
            "de maneira envolvente.\n"
        "4. Garanta que o post esteja estruturado com uma "
            "introdução envolvente, corpo perspicaz "
            "e uma conclusão resumida.\n"
        "5. Revise para corrigir erros gramaticais e "
            "alinhamento com o público-alvo.\n"
    ),
    expected_output="Um post de blog bem escrito, em português do Brasil, "
        "em formato markdown, pronto para publicação, "
        "com cada seção contendo 3 ou 4 parágrafos.",
    agent=escritor,
)


In [12]:
edicao = Task(
    description=("Revise o post de blog para corrigir "
                 "erros gramaticais e "
                 "alinhar com o público-alvo."),
    expected_output="Um post de blog bem escrito em formato markdown, "
                    "pronto para publicação, em portuguê do Brasil,"
                    "com cada seção contendo 3 ou 4 parágrafos.",
    agent=editor
)


In [13]:
crew = Crew(
    agents=[planejador, escritor, editor],
    tasks=[planejamento, escrita, edicao],
    verbose=2
)

In [14]:
result = crew.kickoff(inputs={"tópico": "Seminário de Ensino, Pesquisa e Extensão do IFSC - SEPEI"})

 [DEBUG]: == Working Agent: Planejador de Conteúdo
 [INFO]: == Starting Task: 1. Priorizar as últimas tendências, principais players e notícias relevantes sobre Seminário de Ensino, Pesquisa e Extensão do IFSC - SEPEI.
2. Identificar o público-alvo, considerando seus interesses e pontos de dor.
3. Desenvolver um esboço de conteúdo detalhado, incluindo uma introdução, pontos principais e um chamado para ação.
4. Incluir palavras-chave de SEO e dados ou fontes relevantes.


> Entering new CrewAgentExecutor chain...
Vou começar a coletar informações sobre o Seminário de Ensino, Pesquisa e Extensão do IFSC - SEPEI, focando nas últimas tendências, principais players e notícias relevantes.

Action: Search the internet  
Action Input: {"search_query": "Seminário de Ensino, Pesquisa e Extensão IFSC SEPEI 2023"}   


Search results: Title: Home | SEPEI 2023 - Galoá
Link: https://eventos.galoa.com.br/sepei-2023/page/2381-home
Snippet: O 9º Seminário de Ensino, Pesquisa, Extensão e Inovação do IF